In [1]:
import numpy as np
import pandas as pd

In [31]:
dis_fea = pd.read_csv("../../extract_pubmed/2020_TF-IDF.csv",index_col=0)

# dis_fea["Unnamed: 0"] = dis_fea["Unnamed: 0"].str.lower()
dis_fea.index = dis_fea.index.str.lower()
print(dis_fea.shape)
dis_fea.head()

(4536, 385)


,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"mesothelioma, cystic",0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.0
palatal neoplasms,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.779678,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.0
hemangiosarcoma,0.0,2.779656,1.116918,1.869995,0.0,0.0,0.0,3.898388,0.0,0.0,...,0.0,0.000000,3.01014,5.623722,0.000000,1.078317,0.0,0.0,2.1545,0.0
dna repair-deficiency disorders,0.0,0.000000,3.350754,1.246663,0.0,0.0,0.0,1716.070558,0.0,0.0,...,0.0,0.000000,1.50507,1.124744,1.460402,3.234951,0.0,0.0,0.0000,0.0
"nevus, pigmented",0.0,0.000000,11.727640,3.739990,0.0,0.0,0.0,1.559355,0.0,0.0,...,0.0,1.288102,0.00000,0.000000,0.000000,3.234951,0.0,0.0,0.0000,0.0


###  Construct symptom feature matrix and label

In [32]:
dis_sim = pd.read_csv("../dis_sim.csv")
print(dis_sim.shape)
dis_sim.head()

(44551, 3)


,disease_A,disease_B,s_AB (observed)
0,"abnormalities, multiple",actinomycetales infections,0.810442
1,"abnormalities, multiple",adenocarcinoma,0.306936
2,"abnormalities, multiple",adnexal diseases,0.366111
3,"abnormalities, multiple",adrenal gland diseases,0.649266
4,"abnormalities, multiple",albuminuria,0.212946


In [33]:
#10 differient diseases between old and 2020 version.
dif_dis = pd.read_csv("../../python爬虫/dif_dis.csv",index_col=0)
print(dif_dis)

#Mapping disease from old to 2020 version: changing names
dis_trans = {"metabolic syndrome x":"Metabolic Syndrome".lower(),
            "sleep disorders":"Sleep Wake Disorders".lower()}

                               dif_dis
0                        death, sudden
1                          body weight
2  pathological conditions, anatomical
3                 pathologic processes
4                      sleep disorders
5            urological manifestations
6                 metabolic syndrome x
7                           overweight
8                                death
9                   signs and symptoms


In [90]:
from tqdm import tqdm

disA_fea_mat = [] 
disB_fea_mat = []

disA_name = [] 
disB_name = []

new_label = [] #label

for i in tqdm(range(len(dis_sim))):
    disA = dis_sim.loc[i,"disease_A"]
    disB = dis_sim.loc[i,"disease_B"]
    
    #Skip above different 10 diseases
    if (disA in list(dif_dis["dif_dis"])):
        #2 disease mappings
        if(disA in dis_trans.keys()):
            disA = dis_trans.get(disA)
        else:
            continue
    if (disB in list(dif_dis["dif_dis"])):
        if(disB in dis_trans.keys()):
            disB = dis_trans.get(disB)
        else:
            continue
    
#     print(disA,"\t",disB)
    
    disA_name.append(disA)
    disB_name.append(disB)
    
    disA_fea = dis_fea.loc[disA]
    disB_fea = dis_fea.loc[disB]
    
    disA_fea_mat.append(list(disA_fea))
    disB_fea_mat.append(list(disB_fea))
    
    if dis_sim.loc[i,"s_AB (observed)"]>0:
        new_label.append(0)
    else:
        new_label.append(1)
    

disA_fea_mat = pd.DataFrame(disA_fea_mat,columns=dis_fea.columns)
disA_fea_mat.index = disA_name

disB_fea_mat = pd.DataFrame(disB_fea_mat,columns=dis_fea.columns)
disB_fea_mat.index = disB_name

100%|██████████████████████████████████████████████████████████████████████████| 44551/44551 [00:18<00:00, 2469.07it/s]


AttributeError: 'list' object has no attribute 'value_counts'

#### Positive and negative sample ratio

In [98]:
from collections import Counter
Counter(new_label)

Counter({0: 39438, 1: 2757})

#### 42195 samples/ total 44551

In [35]:
print(disA_fea_mat.shape)
print(len(new_label))

(42195, 385)
42195


In [10]:
disA_fea_mat.head()

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"abnormalities, multiple",87.013803,8.338968,276.995689,44.879875,2.165972,0.0,0.0,170.74941,2.702773,18.81661,...,12.166744,28.338251,21.070979,20.2454,33.589254,39.897729,3.243651,0.0,4.308999,7.103135
"abnormalities, multiple",87.013803,8.338968,276.995689,44.879875,2.165972,0.0,0.0,170.74941,2.702773,18.81661,...,12.166744,28.338251,21.070979,20.2454,33.589254,39.897729,3.243651,0.0,4.308999,7.103135
"abnormalities, multiple",87.013803,8.338968,276.995689,44.879875,2.165972,0.0,0.0,170.74941,2.702773,18.81661,...,12.166744,28.338251,21.070979,20.2454,33.589254,39.897729,3.243651,0.0,4.308999,7.103135
"abnormalities, multiple",87.013803,8.338968,276.995689,44.879875,2.165972,0.0,0.0,170.74941,2.702773,18.81661,...,12.166744,28.338251,21.070979,20.2454,33.589254,39.897729,3.243651,0.0,4.308999,7.103135
"abnormalities, multiple",87.013803,8.338968,276.995689,44.879875,2.165972,0.0,0.0,170.74941,2.702773,18.81661,...,12.166744,28.338251,21.070979,20.2454,33.589254,39.897729,3.243651,0.0,4.308999,7.103135


In [36]:
new_label = pd.DataFrame(new_label,columns=["new_label"])

In [37]:
# disA_fea_mat.to_csv("new_disA_fea_mat.csv")
# disB_fea_mat.to_csv("new_disB_fea_mat.csv")
# new_label.to_csv("new_label.csv")

In [13]:
"death" in list(dif_dis["dif_dis"]) #True
"death" in dif_dis["dif_dis"] #False

False

### Construct Mesh one-hot feature mat

In [38]:
meshid = pd.read_csv("Mesh_files/mtrees_dis_correct.txt",sep = "\t",header = None)
meshid.columns = ["dis_name","mesh_id"]
meshid.head()

,dis_name,mesh_id
0,Infections,C01
1,"Aneurysm, Infected",C01
2,"Arthritis, Infectious",C01
3,"Arthritis, Reactive",C01
4,Asymptomatic Infections,C01


In [39]:
dis_sim = pd.read_csv("../dis_sim.csv")
print(dis_sim.shape)
dis_sim.head()

(44551, 3)


,disease_A,disease_B,s_AB (observed)
0,"abnormalities, multiple",actinomycetales infections,0.810442
1,"abnormalities, multiple",adenocarcinoma,0.306936
2,"abnormalities, multiple",adnexal diseases,0.366111
3,"abnormalities, multiple",adrenal gland diseases,0.649266
4,"abnormalities, multiple",albuminuria,0.212946


#### Match mesh classification to 4536 diseases

In [47]:
print(len(set(dis_fea.index)&set(meshid["dis_name"].str.lower())))

4536


In [75]:
list(set(meshid.loc[meshid["dis_name"]=="Behcet Syndrome"]["mesh_id"]))

['C17', 'C11', 'C14', 'C07', 'C16']

In [76]:
dismesh_dict = dict() 

for dis in dis_fea.index:
    if dis not in dismesh_dict.keys():
        dis_cate_df = meshid.loc[meshid["dis_name"].str.lower()==dis]
        value = list(set(dis_cate_df["mesh_id"]))
        dismesh_dict[dis]= value

print("dict长度：",len(dismesh_dict))

dict长度： 4536


#### Save mesh classification of 4536 diseases
    csv
    josn

In [1]:
dis_mesh = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in dismesh_dict.items()]))

print(dis_mesh.shape)
dis_mesh

NameError: name 'pd' is not defined

In [80]:
# dis_mesh.to_csv("Mesh_files/dis_mesh_new.csv")

In [81]:
"""2、dict to json"""
import json

with open('Mesh_files/dis_mesh_new.json','a') as outfile:
    json.dump(dismesh_dict,outfile,ensure_ascii=False)
    outfile.write('\n')

#### construct mesh one-hot

In [84]:
import json
f=open("Mesh_files/dis_mesh_new.json","r")
dict_data=list()

for line in f:
    dict_data.append(json.loads(line))
f.close()

print(type(dict_data[0]))
print(dict_data[0]["stroke"]) #data[0]-dict

<class 'dict'>
['C14', 'C10']


In [85]:
mul_labels =[] #classifications [{},{},{}]
dis=[] 

for k, v in dict_data[0].items():
    dis.append(k.lower())
    #print(type(k),type(v)) str,list
    mul_labels.append(set(v))
    
print(len(mul_labels),len(dis))
mul_labels[:2]

4536 4536


[{'C04'}, {'C04', 'C05', 'C07'}]

In [87]:
"""label_array :classification label
"""
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
label_array = mlb.fit_transform(mul_labels)

print(len(label_array[0]))
label_array[0]

24


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [88]:
# pd.DataFrame(label_array,index=dis).to_csv("Mesh_files/label_array_new.csv")

##### construct one-hot mat

In [102]:
mesh_arr = pd.read_csv("Mesh_files/label_array_new.csv",index_col=0)
print(mesh_arr.shape)
mesh_arr[:2]

(4536, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
"mesothelioma, cystic",0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
palatal neoplasms,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
from tqdm import tqdm

disA_onehot = []
disB_onehot = []

disA_name = [] 
disB_name = []

for i in tqdm(range(len(dis_sim))):
    disA = dis_sim.loc[i,"disease_A"]
    disB = dis_sim.loc[i,"disease_B"]
    
    if (disA in list(dif_dis["dif_dis"])):
        if(disA in dis_trans.keys()):
            disA = dis_trans.get(disA)
        else:
            continue
    if (disB in list(dif_dis["dif_dis"])):
        if(disB in dis_trans.keys()):
            disB = dis_trans.get(disB)
    
#     print(disA,"\t",disB)
    
    disA_name.append(disA)
    disB_name.append(disB)
    
    disA_fea = mesh_arr.loc[disA]
    disB_fea = mesh_arr.loc[disB]
    
    disA_onehot.append(list(disA_fea))
    disB_onehot.append(list(disB_fea))
    
disA_onehot = pd.DataFrame(disA_onehot)
disA_onehot.index = disA_name

disB_onehot = pd.DataFrame(disB_onehot)
disB_onehot.index = disB_name

100%|██████████████████████████████████████████████████████████████████████████| 44551/44551 [00:13<00:00, 3396.81it/s]


In [106]:
print(disA_onehot.shape)
disA_onehot.head()

(42195, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
"abnormalities, multiple",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"abnormalities, multiple",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"abnormalities, multiple",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"abnormalities, multiple",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"abnormalities, multiple",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
# disA_onehot.to_csv("concat_mesh_label/mesh_label_A_new.csv")
# disB_onehot.to_csv("concat_mesh_label/mesh_label_B_new.csv")